In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import visdom
vis = visdom.Visdom()

Setting up a new session...


In [2]:
def loss_tracker(loss_plot, loss_value, num):
    vis.line(Y=loss_value, X=num, win=loss_plot, update='append')

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

#hyperparameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

#load data
mnist_train = dsets.MNIST(root="MNIST_data/", train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dsets.MNIST(root="MNIST_data/", train=False, transform=transforms.ToTensor(), download=True)
dataloader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size, 
                                         shuffle=True, drop_last=True)

In [4]:
#Define CNN
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc1 = nn.Linear(3*3*128, 625, bias=True)
        self.fc2 = nn.Linear(625, 10)
        self.relu = nn.ReLU()
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [5]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [6]:
#make plot
loss_plt = vis.line(Y=torch.Tensor(1).zero_(), opts=dict(title='loss_tracker', legend=['loss'], 
                                                        showlegend=True))

In [7]:
#training
total_batch = len(dataloader)
model.train()
for epoch in range(training_epochs):
    avg_cost = 0
    for X, Y in dataloader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        avg_cost += cost/total_batch
    print("Epoch:{} cost:{}".format(epoch+1, avg_cost))
    loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))

Epoch:1 cost:0.1598575860261917
Epoch:2 cost:0.042148664593696594
Epoch:3 cost:0.03047478199005127
Epoch:4 cost:0.02173825539648533
Epoch:5 cost:0.019176989793777466
Epoch:6 cost:0.013583950698375702
Epoch:7 cost:0.013751080259680748
Epoch:8 cost:0.010209211148321629
Epoch:9 cost:0.009185529313981533
Epoch:10 cost:0.008032329380512238
Epoch:11 cost:0.008217612281441689
Epoch:12 cost:0.007303837686777115
Epoch:13 cost:0.005733550060540438
Epoch:14 cost:0.006129049696028233
Epoch:15 cost:0.0042299386113882065


In [8]:
#testing
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction,1) == Y_test
    accuracy = correct_prediction.float().mean()
    print("Accuracy:", accuracy.item())

C:\anaconda3\envs\tensorflow\lib\site-packages\torchvision\datasets\mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\anaconda3\envs\tensorflow\lib\site-packages\torchvision\datasets\mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9914000034332275


[WinError 10054] 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다
on_close() takes 1 positional argument but 3 were given
[WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다
on_close() takes 1 positional argument but 3 were given
